In [ ]:
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
!pip install pad_sequences

In [ ]:
df = pd.read_csv('datasets/preprocessed_sentiment.csv', usecols=['tweets','labels'])

In [ ]:
# check if there is any NaN value

df.tweets.isnull().values.any()
df.tweets.isnull().sum()

In [ ]:
# drop the NaN values if any

df = df.dropna()
df.labels.value_counts() 

### CNN with Keras embedding

In [ ]:
df.tweets = df.tweets.map(lambda x:str(x))

In [ ]:
x_train, xtest, y_train, ytest = train_test_split(df.tweets.values, df.labels.values,stratify=df.labels.values, test_size=0.3,random_state=1)
y_train = to_categorical(y_train)

In [ ]:
 xtest.size

In [ ]:
 xtest.size*0.3

In [ ]:
x_val = xtest[0:14885]
y_val = to_categorical(ytest[0:14885])
x_test = xtest[14885:]
y_test = ytest[14885:]

In [ ]:
def cnn_keras(activation,optimizer,epochs,batchsize):
    embedding_dim = 100

    model = Sequential()
    model.add(layers.Embedding(input_dim=vocab_size, 
                          output_dim=embedding_dim,
                          input_length=maxlen))

    model.add(Conv1D(32, kernel_size=2, activation=activation))
    model.add(layers.MaxPooling1D(2))
    model.add(Conv1D(32, kernel_size=2, activation=activation))
    model.add(Dropout(0.25))
    model.add(Conv1D(16, kernel_size=2, activation=activation))
    model.add(layers.MaxPooling1D(2))
    model.add(Conv1D(16, kernel_size=2, activation=activation))
    model.add(Dropout(0.25))

    model.add(Flatten())

    model.add(Dense(64, activation='tanh'))
    model.add(Dropout(0.5))

    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer=optimizer,
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    
    history = model.fit(X_train, y_train,
                    epochs=epochs,
                    verbose=0,
                    validation_data=(X_val, y_val),
                    batch_size=batchsize)

    return history, model

In [ ]:
# Best model selection
sel_activation_final = "elu"
sel_optimizer_final = "adam"
sel_epoch_final = 10
sel_batch_final = 16

history, model = cnn_keras(sel_activation_final,sel_optimizer_final,sel_epoch_final,sel_batch_final)
pred = np.argmax(model.predict(X_test), axis=-1)
print("test accuracy score = ",accuracy_score(y_pred=pred, y_true=y_test))
print("test f1 score = ",f1_score(y_pred=pred, y_true=y_test, average="weighted"))

In [ ]:
model.save('/content/drive/MyDrive/computational-social-media/cnnkeras.h5')